In [2]:
from pymongo import MongoClient
import configparser
import pandas as pd
import re
from nltk.corpus import wordnet
import numpy as np
from flask import Flask, render_template
# import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

### 資料處理

In [12]:
_df = pd.read_csv("./data.csv")

In [4]:
# 建立字典，將年月份對應到角度
month_labels = ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
               '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01',
               '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01']
angle_mapping = [(i * 15) % 360 for i in range(len(month_labels))]
dict_date_angle = dict(zip(month_labels, angle_mapping))


In [6]:
def transform(text):
    text = str(text)
    year = text[:4]
    month = text[-2:]
    return year + "-" + month

In [7]:
# 將日期轉換為角度
def date_to_angle(date):
    date = date.strftime("%Y-%m-%d")
    return dict_date_angle[date]

In [14]:
_df['monitormonth'] = pd.to_datetime(_df["monitormonth"].apply(lambda x: transform(x)), format='%Y-%m')

In [16]:
_df

,siteid,sitename,itemid,itemname,itemengname,itemunit,monitormonth,concentration
0,6,板橋,1,二氧化硫,SO2,ppb,1982-07-01,40.6
1,6,板橋,38,相對濕度,RH,percent,1982-07-01,15.86
2,67,三重,5,氮氧化物,NOx,ppb,1982-07-01,189.54
3,6,板橋,9,非甲烷碳氫化合物,NMHC,ppm,1982-07-01,x
4,6,板橋,7,二氧化氮,NO2,ppb,1982-07-01,23
...,...,...,...,...,...,...,...,...
366765,85,大城,6,一氧化氮,NO,ppb,2023-04-01,1.1
366766,85,大城,7,二氧化氮,NO2,ppb,2023-04-01,6.8
366767,85,大城,8,總碳氫化物,THC,ppm,2023-04-01,2.22
366768,85,大城,9,非甲烷碳氫化合物,NMHC,ppm,2023-04-01,0.01


In [17]:
# find the min of monitormonth
min_date = _df['monitormonth'].min()
max_date = _df['monitormonth'].max()
print(min_date.strftime('%Y-%m-%d'), "~", max_date.strftime('%Y-%m-%d'))

1982-07-01 ~ 2023-04-01


In [18]:
start_date = pd.to_datetime('2021-01', format='%Y-%m')
end_date = pd.to_datetime('2022-12', format='%Y-%m')
filtered_df = _df[(_df['monitormonth'] >= start_date) & (_df['monitormonth'] <= end_date)]

In [21]:
filtered_df['sitename'].unique().tolist()

['富貴角',
 '麥寮',
 '關山',
 '馬公',
 '金門',
 '馬祖',
 '埔里',
 '復興',
 '永和',
 '竹山',
 '中壢',
 '三重',
 '冬山',
 '宜蘭',
 '陽明',
 '花蓮',
 '臺東',
 '恆春',
 '潮州',
 '屏東',
 '小港',
 '前鎮',
 '前金',
 '左營',
 '楠梓',
 '林園',
 '大寮',
 '鳳山',
 '仁武',
 '橋頭',
 '美濃',
 '臺南',
 '安南',
 '善化',
 '新營',
 '嘉義',
 '臺西',
 '朴子',
 '新港',
 '崙背',
 '斗六',
 '南投',
 '二林',
 '線西',
 '彰化',
 '西屯',
 '忠明',
 '大里',
 '沙鹿',
 '豐原',
 '三義',
 '苗栗',
 '頭份',
 '新竹',
 '竹東',
 '湖口',
 '龍潭',
 '平鎮',
 '觀音',
 '大園',
 '桃園',
 '大同',
 '松山',
 '古亭',
 '萬華',
 '中山',
 '士林',
 '淡水',
 '林口',
 '菜寮',
 '新莊',
 '板橋',
 '土城',
 '新店',
 '萬里',
 '汐止',
 '基隆',
 '大城']

In [50]:
loc = filtered_df[filtered_df['sitename']=="埔里"]

In [55]:
loc

,siteid,sitename,itemid,itemname,itemengname,itemunit,monitormonth,concentration
336080,72,埔里,143,小時風速值,WS_HR,m/sec,2021-01-01,1
336081,72,埔里,38,相對濕度,RH,%,2021-01-01,72
336082,72,埔里,33,細懸浮微粒,PM2.5,μg/m3,2021-01-01,22
336083,72,埔里,14,溫度,AMB_TEMP,℃,2021-01-01,15.2
336084,72,埔里,10,風速,WIND_SPEED,m/sec,2021-01-01,1.4
...,...,...,...,...,...,...,...,...
362277,72,埔里,14,溫度,AMB_TEMP,℃,2022-12-01,18
362278,72,埔里,31,甲烷,CH4,ppm,2022-12-01,2.1
362279,72,埔里,33,細懸浮微粒,PM2.5,μg/m3,2022-12-01,12
362280,72,埔里,38,相對濕度,RH,%,2022-12-01,69


In [56]:
pd.pivot_table(loc, values='concentration', index='monitormonth', columns='itemname')

itemname,一氧化氮,一氧化碳,二氧化氮,二氧化硫,小時風速值,懸浮微粒,氮氧化物,溫度,甲烷,相對濕度,細懸浮微粒,總碳氫化物,臭氧,非甲烷碳氫化合物,風速
monitormonth,,,,,,,,,,,,,,,
2021-01-01,2.6,0.41,11.7,1.4,1.0,31.0,14.3,15.2,NaN,72.0,22.0,NaN,26.3,NaN,1.4
2021-02-01,1.9,0.39,11.4,1.5,1.1,39.0,13.3,18.5,NaN,67.0,26.0,NaN,36.8,NaN,1.6
2021-03-01,1.8,0.38,10.9,1.5,1.3,39.0,12.7,21.6,NaN,71.0,25.0,NaN,35.2,NaN,1.7
2021-04-01,1.7,0.34,9.9,1.5,1.3,38.0,11.6,23.6,NaN,70.0,24.0,NaN,79.9,NaN,1.7
2021-05-01,2.0,0.21,9.1,1.4,1.4,24.0,11.1,27.4,NaN,69.0,13.0,NaN,31.6,NaN,1.9
2021-06-01,1.6,0.16,5.3,1.3,0.9,16.0,7.0,25.8,NaN,80.0,5.0,NaN,22.4,NaN,1.4
2021-07-01,1.1,0.14,3.9,1.1,1.2,15.0,5.2,27.4,NaN,76.0,8.0,NaN,26.0,NaN,1.6
2021-08-01,1.4,0.16,4.4,1.1,0.9,14.0,5.9,26.2,NaN,82.0,8.0,NaN,20.8,NaN,1.2
2021-09-01,1.4,0.21,4.9,1.3,1.2,22.0,6.4,27.2,NaN,75.0,14.0,NaN,31.9,NaN,1.6


In [63]:
loc = filtered_df[filtered_df['sitename']=="臺南"]
loc = loc.replace('x', 0)
loc = loc.fillna(0)
loc = pd.pivot_table(loc, values='concentration', index=loc['monitormonth'], columns='itemname')
loc = loc.reset_index()
df_location = loc[['monitormonth', '細懸浮微粒', '懸浮微粒', '一氧化氮', '相對濕度']]
df_location['angle'] = df_location.monitormonth.apply(lambda x: date_to_angle(x))

c:\Users\User\anaconda3\envs\text\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [65]:
_df.sitename.unique().tolist()

['板橋',
 '三重',
 '前金',
 '復興',
 '鳳山',
 '永和',
 '中壢',
 '臺南',
 '松山',
 '花蓮',
 '基隆',
 '楠梓',
 '新竹',
 '嘉義',
 '彰化',
 '頭份',
 '汐止',
 '林園',
 '斗六',
 '古亭',
 '大同',
 '線西',
 '平鎮',
 '朴子',
 '新港',
 '觀音',
 '橋頭',
 '恆春',
 '小港',
 '大寮',
 '崙背',
 '臺西',
 '新營',
 '桃園',
 '臺東',
 '冬山',
 '湖口',
 '潮州',
 '苗栗',
 '前鎮',
 '三義',
 '仁武',
 '竹東',
 '善化',
 '西屯',
 '大園',
 '左營',
 '二林',
 '龍潭',
 '安南',
 '屏東',
 '忠明',
 '南投',
 '陽明',
 '大里',
 '沙鹿',
 '美濃',
 '宜蘭',
 '豐原',
 '淡水',
 '林口',
 '萬華',
 '菜寮',
 '新莊',
 '土城',
 '士林',
 '中山',
 '萬里',
 '新店',
 '竹山',
 '埔里',
 '馬祖',
 '金門',
 '馬公',
 '關山',
 '麥寮',
 '富貴角',
 '大城']

In [ ]:
central = ['大城','埔里','竹山','南投','二林','線西','彰化','西屯','忠明','大里','沙鹿','豐原']


In [64]:
df_location

itemname,monitormonth,細懸浮微粒,懸浮微粒,一氧化氮,相對濕度,angle
0,2021-01-01,37.0,60.0,3.0,72.0,0
1,2021-02-01,41.0,69.0,1.6,71.0,15
2,2021-03-01,37.0,69.0,2.0,71.0,30
3,2021-04-01,24.0,56.0,1.1,70.0,45
4,2021-05-01,11.0,25.0,1.3,71.0,60
5,2021-06-01,6.0,15.0,1.2,78.0,75
6,2021-07-01,7.0,17.0,1.3,78.0,90
7,2021-08-01,7.0,15.0,1.6,87.0,105
8,2021-09-01,13.0,25.0,1.0,80.0,120
9,2021-10-01,16.0,38.0,1.3,72.0,135


In [39]:
import plotly.graph_objects as go

# 自定義的年月份標籤
month_labels = ['2021/01', '2021/02', '2021/03', '2021/04', '2021/05', '2021/06',
               '2021/07', '2021/08', '2021/09', '2021/10', '2021/11', '2021/12',
               '2022/01', '2022/02', '2022/03', '2022/04', '2022/05', '2022/06',
               '2022/07', '2022/08', '2022/09', '2022/10', '2022/11', '2022/12']

# 創建 Polar Charts
fig = go.Figure()

# 添加數據到圖表，並設定標籤文本
# fig.add_trace(go.Scatterpolar(
#     name='相對濕度',
#     r=df_location['相對濕度'].tolist(),
#     theta=month_labels,
#     mode='markers',  # 使用 'lines+markers' 顯示線段和點
#     marker=dict(
#         size=10,
#         color=df_location['相對濕度'].tolist(),  # 根據 r 值設定顏色
#         colorscale='Blues',  # 設定顏色漸層
#         # colorbar=dict(
#         #     title='相對濕度'  # 設定顏色軸標題
#         # ),
#         cmin=0.9*min(df_location['相對濕度'].tolist()),  # 設定顏色漸層最小值
#         cmax=max(df_location['相對濕度'].tolist())  # 設定顏色漸層最大值
#     ),
    
#     line=dict(
#         color='blue',
#         width=1
#     ),
#     text=month_labels,  # 設定標籤文本為年月份
#     hovertemplate="日期: %{text}<br>percent: %{r} %<br>",  # 定義 hover 顯示的文本格式
# ))
fig.add_trace(go.Scatterpolar(
    name='細懸浮微粒',
    r=df_location['相對濕度'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        color=df_location['細懸浮微粒'].tolist(),  # 根據 r 值設定顏色
        # symbol=df_location['細懸浮微粒'].tolist(),
        colorscale='Reds',  # 設定顏色漸層
        size = 10,
        colorbar=dict(
            title='細懸浮微粒'  # 設定顏色軸標題
        ),
        cmin=0.9*min(df_location['細懸浮微粒'].tolist()),  # 設定顏色漸層最小值
        cmax=max(df_location['細懸浮微粒'].tolist())  # 設定顏色漸層最大值
    ),
    line=dict(
        color='blue',
        width=1
    ),
    # text=month_labels,  # 設定標籤文本為年月份
    # hovertemplate="日期: %{text}<br>μg/m3: %{r}<br>",  # 定義 hover 顯示的文本格式
))

# 設定角度刻度和標籤，並且顏色填充用藍色根據r值由小到大漸層
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            angle=0,
            tickvals=angle_mapping,
            ticks="outside",
            tickmode='array',
            tickfont=dict(
                size=8,
            )
        )
    ),
    height=600,
    # width=600,
    # showlegend=False
)

# 顯示圖表
fig.show()

In [1]:
import plotly.graph_objects as go

# 自定義的年月份標籤
month_labels = ['2021/01', '2021/02', '2021/03', '2021/04', '2021/05', '2021/06',
               '2021/07', '2021/08', '2021/09', '2021/10', '2021/11', '2021/12',
               '2022/01', '2022/02', '2022/03', '2022/04', '2022/05', '2022/06',
               '2022/07', '2022/08', '2022/09', '2022/10', '2022/11', '2022/12']

# 創建 Polar Charts
fig = go.Figure()

# 添加數據到圖表，並設定標籤文本
fig.add_trace(go.Scatterpolar(
    name='相對濕度',
    r=df_location['相對濕度'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        size=10,
        color=df_location['相對濕度'].tolist(),  # 根據 r 值設定顏色
        colorscale='Blues',  # 設定顏色漸層
        colorbar=dict(
            title='相對濕度',
            len=0.4,
            y=0.25,
            x=1.1,
            tickfont=dict(
                size=9,
            ),
            xpad=2
        ),
        cmin=0.9*min(df_location['相對濕度'].tolist()),  # 設定顏色漸層最小值
        cmax=max(df_location['相對濕度'].tolist())  # 設定顏色漸層最大值
    ),
    
    line=dict(
        color='blue',
        width=1
    ),
    text=month_labels,  # 設定標籤文本為年月份
    hovertemplate="日期: %{text}<br>percent: %{r} %<br>",  # 定義 hover 顯示的文本格式
))
fig.add_trace(go.Scatterpolar(
    name='細懸浮微粒',
    r=df_location['細懸浮微粒'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        size=10,
        color=df_location['細懸浮微粒'].tolist(),  # 根據 r 值設定顏色
        colorscale='Reds',  # 設定顏色漸層,
        colorbar=dict(
            title='細懸浮微粒',
            len=0.4,
            y=0.25,
            x=0.9,
            tickfont=dict(
                size=9,
            ),
            xpad=2
        ),
        cmin=0.9*min(df_location['細懸浮微粒'].tolist()),  # 設定顏色漸層最小值
        cmax=max(df_location['細懸浮微粒'].tolist())  # 設定顏色漸層最大值
    ),
    line=dict(
        color='blue',
        width=1
    ),
    text=month_labels,  # 設定標籤文本為年月份
    hovertemplate="日期: %{text}<br>μg/m3: %{r}<br>",  # 定義 hover 顯示的文本格式
))
fig.add_trace(go.Scatterpolar(
    name='懸浮微粒',
    r=df_location['懸浮微粒'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        size=10,
        color=df_location['懸浮微粒'].tolist(),  # 根據 r 值設定顏色
        colorscale='Greens',  # 設定顏色漸層
        colorbar=dict(
            title='懸浮微粒',
            len=0.4,
            y=0.25,
            x=1,
            tickfont=dict(
                size=9,
            ),
            xpad=2
        ),
        cmin=min(df_location['懸浮微粒'].tolist()),  # 設定顏色漸層最小值
        cmax=0.9*max(df_location['懸浮微粒'].tolist())  # 設定顏色漸層最大值
    ),
    line=dict(
        color='blue',
        width=1
    ),
    text=month_labels,  # 設定標籤文本為年月份
    hovertemplate="日期: %{text}<br>μg/m3: %{r}<br>",  # 定義 hover 顯示的文本格式 
))

# 設定角度刻度和標籤，並且顏色填充用藍色根據r值由小到大漸層
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            angle=0,
            tickvals=angle_mapping,
            ticks="outside",
            tickmode='array',
            tickfont=dict(
                size=8,
            )
        )
    ),
    height=600,
    # coloraxis_colorbar=colorbar
)

# 顯示圖表
fig.show()

NameError: name 'df_location' is not defined

In [ ]:
import plotly.graph_objects as go

# 設定角度刻度和標籤
angles = [i * 15 for i in range(24)]  # 每十五度一個刻度
labels = [str(i) + "°" for i in angles]  # 刻度標籤

# 創建 Polar Chart
fig = go.Figure()

# 添加從中心往外的線
for angle in angles:
    fig.add_trace(go.Scatterpolar(
        r=[0, 1],  # 從中心 (r=0) 繪製到半徑為 1 的線
        theta=[angle, angle],  # 角度起點和終點相同，畫一條直線
        mode='lines',
        line=dict(
            color='black',
            width=1
        ),
        hoverinfo='skip'  # 不顯示 hover 資訊
    ))

# 設定角度刻度和標籤
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=False  # 隱藏半徑軸
        ),
        angularaxis=dict(
            tickvals=angles,
            ticktext=labels,
            tickmode='array',
            tickfont=dict(
                size=8,
            )
        )
    )
)

# 顯示圖表
fig.show()
